In [1]:
import pandas as pd
import numpy as np
import scipy
from matplotlib import pyplot as plt
from scipy import stats
import math
import scipy.optimize
from New_Function_set import *
import tqdm
import sys
import os
import pickle
import copy

In [2]:
df_breast = pd.read_csv('/Users/mhy/Desktop/Project/AlexandrovEtAI/somatic_mutation_data/Breast/Breast_raw_mutations_data.txt',\
                        delimiter = "\t", header=None)

In [3]:
df_breast

,0,1,2,3,4,5,6,7
0,PD3851a,indel,10,60723391,60723409,CAGAGATTGCAGGAAAAAA,-,ORIGINAL-DATA
1,PD3851a,indel,1,220948160,220948161,TG,-,ORIGINAL-DATA
2,PD3851a,indel,12,21374558,21374561,ACAA,-,ORIGINAL-DATA
3,PD3851a,indel,12,59999008,59999008,C,-,ORIGINAL-DATA
4,PD3851a,indel,14,52111863,52111865,TTC,-,ORIGINAL-DATA
...,...,...,...,...,...,...,...,...
736445,TCGA-GI-A2C8-01A-11D-A16D-09,subs,X,18961899,18961899,C,A,ORIGINAL-DATA
736446,TCGA-GI-A2C8-01A-11D-A16D-09,subs,X,24382956,24382956,G,A,ORIGINAL-DATA
736447,TCGA-GI-A2C8-01A-11D-A16D-09,subs,X,49074448,49074448,A,T,ORIGINAL-DATA
736448,TCGA-GI-A2C8-01A-11D-A16D-09,subs,X,53966853,53966853,T,A,ORIGINAL-DATA


In [3]:
def FitWithRecord(df, distance, c, threshold_list=[]):
    sliding_time = int(len(distance))
    dist_exp = getattr(stats, 'expon')
    dist_powerlaw = getattr(stats, 'powerlaw')
    coefficient_dict = {}
    coefficient_dict_new = {}
    
    for i in range(sliding_time):
        threshold = threshold_list[i]
        length = threshold/100
        x_, y_, x_n, y_n = hist_creat(df, i, c, 'short', threshold, length)
        x_l, y_l, x_nl, y_nl = hist_creat(df, i, c, 'long', threshold, length)

        y_raw = [x for x in distance[i] if (math.isnan(x) == False) & (x <= threshold)]
        y_rawl = [x for x in distance[i] if (math.isnan(x) == False) & (x > threshold)]

        x_n, y_n = del_zero(x_n, y_n)
        x_nl, y_nl = del_zero(x_nl, y_nl)
        x_, y_ = del_zero(x_, y_)
        x_l, y_l = del_zero(x_l, y_l)
        
        ### for short distance ###
        short_dict = {}
        if len(y_raw)==0:
            indicator = 2
        else:
            param_exp = dist_exp.fit(y_raw)
            D_exp, p_exp = stats.kstest(y_raw, 'expon', args=param_exp)
            param_law = dist_powerlaw.fit(y_raw)
            D_law, p_law = stats.kstest(y_raw, 'powerlaw', args=param_law)
            indicator = list([D_exp, D_law]).index(min(D_exp, D_law))
        
        if indicator == 0:  ### expon
            if len(y_) <=1:
                short_dict['expon'] = ([np.nan, np.nan], np.nan)
            else:
                if len(y_n)<=1:
                    params_, cv_ = scipy.optimize.curve_fit(linear_function, x_, np.log10(y_))
                    std_er=sum(np.square(np.log10(y_)-linear_function(params_[0],np.array(x_),params_[1])))/len(x_)
                    std_er = np.sqrt(std_er)
                    short_dict['expon'] = (params_, std_er)
                else:
                    params_, cv_ = scipy.optimize.curve_fit(linear_function, x_n, np.log10(y_n))
                    std_er=sum(np.square(np.log10(y_n)-linear_function(params_[0],np.array(x_n),params_[1])))/len(x_n)
                    std_er = np.sqrt(std_er)
                    short_dict['expon'] = (params_, std_er)

        if indicator == 1: ### power-law
            if len(y_) <=1:
                short_dict['powerlaw'] = ([np.nan, np.nan], np.nan)
            else:
                if len(y_n)<=1:
                    params_, cv_ = scipy.optimize.curve_fit(linear_function, np.log10(x_), np.log10(y_))
                    std_er=sum(np.square(np.log10(y_)-linear_function(params_[0],np.array(np.log10(x_)),params_[1])))/len(x_)
                    std_er = np.sqrt(std_er)
                    short_dict['powerlaw'] = (params_, std_er)
                else:
                    params_, cv_ = scipy.optimize.curve_fit(linear_function, np.log10(x_n), np.log10(y_n))
                    std_er=sum(np.square(np.log10(y_n)-linear_function(params_[0],np.array(np.log10(x_n)),params_[1])))/len(x_n)
                    std_er = np.sqrt(std_er)
                    short_dict['powerlaw'] = (params_, std_er)

        if indicator == 2:
            short_dict['empty'] = ([np.nan, np.nan], np.nan)
            
        ### for long distance ###
        long_dict = {}
        if len(y_rawl)==0:
            indicator = 2
        else:
            param_exp = dist_exp.fit(y_rawl)
            D_exp, p_exp = stats.kstest(y_rawl, 'expon', args=param_exp)
            param_law = dist_powerlaw.fit(y_rawl)
            D_law, p_law = stats.kstest(y_rawl, 'powerlaw', args=param_law)
            indicator = list([D_exp, D_law]).index(min(D_exp, D_law))

        if indicator == 0:  ### expon
            if len(y_l) <=1:
                long_dict['expon'] = ([np.nan, np.nan], np.nan)
            else:
                if len(y_nl)<=1:
                    params_, cv_ = scipy.optimize.curve_fit(linear_function, x_l, np.log10(y_l))
                    std_er=sum(np.square(np.log10(y_l)-linear_function(params_[0],np.array(x_l),params_[1])))/len(x_l)
                    std_er = np.sqrt(std_er)
                    long_dict['expon'] = (params_, std_er)
                else:
                    params_, cv_ = scipy.optimize.curve_fit(linear_function, x_nl, np.log10(y_nl))
                    std_er=sum(np.square(np.log10(y_nl)-linear_function(params_[0],np.array(x_nl),params_[1])))/len(x_nl)
                    std_er = np.sqrt(std_er)
                    long_dict['expon'] = (params_, std_er)

        if indicator == 1:  ### powerlaw
            if len(y_l) <=1:
                long_dict['powerlaw'] = ([np.nan, np.nan], np.nan)
            else:
                if len(y_nl)<=1:
                    params_, cv_ = scipy.optimize.curve_fit(linear_function, np.log10(x_l), np.log10(y_l))
                    std_er=sum(np.square(np.log10(y_l)-linear_function(params_[0],np.array(np.log10(x_l)),params_[1])))/len(x_l)
                    std_er = np.sqrt(std_er)
                    long_dict['powerlaw'] = (params_, std_er)
                else:
                    params_, cv_ = scipy.optimize.curve_fit(linear_function, np.log10(x_nl), np.log10(y_nl))
                    std_er=sum(np.square(np.log10(y_nl)-linear_function(params_[0],np.array(np.log10(x_nl)),params_[1])))/len(x_nl)
                    std_er = np.sqrt(std_er)
                    long_dict['powerlaw'] = (params_, std_er)

        if indicator == 2:
            long_dict['empty'] = ([np.nan, np.nan], np.nan)
            
        coefficient_dict[i] = (short_dict, long_dict)
        
    return coefficient_dict

In [4]:
def calculations(df, patient_id, distance, df_bucket, c_id): # df is the readed csv file.
    threshold_list = []
    for i in range(0, 29):
        threshold_can, dict_can = maxlikelihood(distance[c_id[3:]], i, range(3000, 10000, 100))
        threshold_list = threshold_list + [int(threshold_can)] # belike [[1theta],[2theta],...,[29theta]]
    filter_col = [s for s in list(df_bucket.columns) if s.endswith(c_id)]
    record_dict = FitWithRecord(df_bucket[filter_col], distance[c_id[3:]], c_id[3:],threshold_list)
    
    return record_dict, threshold_list

In [6]:
patient_id = 'PD4107a'
df_1 = df_breast[(df_breast[0] == patient_id) & (df_breast[1]=='subs')]
distance, df_bucket = window_sliding(df_1,2,3,4,50000000,7000000,autosomal=True)

In [9]:
distance.keys()

dict_keys(['20', '1', '9', '11', '15', '10', '12', '4', '7', '3', '5', '2', '19', '14', '18', '16', '13', '8', '17', '21', '22', '6'])

In [7]:
df_bucket

,distance_0chr20,counts_0chr20,distance_0chr1,counts_0chr1,distance_0chr9,counts_0chr9,distance_0chr11,counts_0chr11,distance_0chr15,counts_0chr15,...,distance_28chr8,counts_28chr8,distance_28chr17,counts_28chr17,distance_28chr21,counts_28chr21,distance_28chr22,counts_28chr22,distance_28chr6,counts_28chr6
0,1.0,2.0,192998.0,1.0,350881.0,1.0,1.0,4.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,2.0,686401.0,1.0,263776.0,1.0,37550.0,1.0,88607.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55632.0,1.0,158976.0,1.0,509035.0,1.0,1896638.0,1.0,122936.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,137822.0,1.0,83784.0,1.0,26690.0,1.0,73917.0,1.0,105783.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,446127.0,1.0,609868.0,1.0,1521219.0,1.0,359082.0,1.0,11312.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
patient_id = 'PD4107a'
df_1 = df_breast[(df_breast[0] == patient_id) & (df_breast[1]=='subs')]
distance, df_bucket = lambda_sliding(df_1,2,3,4,50000000,7000000,autosomal=True)
record_dict, threshold_list = calculations(df_breast,'PD4107a', distance, df_bucket, 'chr6')

/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1760: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/mhy/Library/Python/3.7/lib/python/site-pac

2
2


/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


In [53]:
record_dict

{0: ({'powerlaw': (array([-0.32125907,  1.16890521]), 0.04742939351431067)},
  {'expon': (array([-3.69835674e-07,  5.47381262e-01]), 0.17701086160806853)}),
 1: ({'powerlaw': (array([-0.37297023,  1.35705693]), 0.05506382066217318)},
  {'expon': (array([-3.98651583e-07,  5.92229940e-01]), 0.18849200420262494)}),
 2: ({'powerlaw': (array([-0.31759513,  1.16193177]), 0.04176685917469377)},
  {'expon': (array([-6.19802212e-07,  6.44123142e-01]), 0.2097316156629595)}),
 3: ({'powerlaw': (array([-0.31272949,  1.06328925]), 0.011937831127169226)},
  {'expon': (array([-1.09121118e-06,  8.12148826e-01]), 0.22089497608761807)}),
 4: ({'powerlaw': (array([-1.26185951,  3.12027123]), 7.850462293418876e-17)},
  {'expon': (array([-8.07895245e-07,  7.16782153e-01]), 0.27851166626135776)}),
 5: ({'powerlaw': (array([-0.68260619,  1.70536226]), 3.925231146709438e-17)},
  {'expon': (array([-1.30111094e-06,  8.45867429e-01]), 0.27034532835701836)}),
 6: ({'powerlaw': (array([-0.5       ,  1.21568188]), 

In [87]:
k_dict = {'PD7409a':['chr8'], 'PD7404a':['chr17','chr1'], 'PD4199a':['chr5','chr8','chr1'],\
          'PD4103a':['chr3','chr4','chr8','chr12'], 'PD4107a':['chr6']}

Nk_dict = {'PD6409a':['chr10'], 'PD6043a':['chr6'], 'PD4109a':['chr5'], 'PD4224a':['chr6'],\
          'PD4072a':['chr12'], 'PD4955a':['chr3'], 'PD4607a':['chr5'], 'PD4120a':['chr8']\
         }

In [88]:
K_record_dict = {}

for patient in tqdm.tqdm(k_dict.keys()):
    K_record_dict[patient] = {}
    patient_id = patient
    df_1 = df_breast[(df_breast[0] == patient_id) & (df_breast[1]=='subs')]
    print('=======> now is sliding <=======')
    distance, df_bucket = lambda_sliding(df_1,2,3,4,50000000,7000000,autosomal=True)
    for c in k_dict[patient_id]:
        K_record_dict[patient][c] = {}
        print('=======> now is fitting <=======')
        record_dict, threshold_list = calculations(df_breast, patient_id, distance, df_bucket, c)
        K_record_dict[patient][c]['record'] = record_dict
        K_record_dict[patient][c]['theta'] = threshold_list

  0%|                                                     | 0/5 [00:00<?, ?it/s]

=======> now is sliding <=======
=======> now is fitting <=======


/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1760: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/mhy/Library/Python/3.7/lib/python/site-pac

=======> now is sliding <=======
=======> now is fitting <=======


/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1760: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


...

/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1760: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


=======> now is fitting <=======


/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1760: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=d

=======> now is sliding <=======
=======> now is fitting <=======


/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1760: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/mhy/Library/Python/3.7/lib/python/site-pac

In [89]:
f = open('/Users/mhy/Desktop/Project/AlexandrovEtAI/Round2Results/Kataegis_StdEr.pkl', 'wb')
pickle.dump(K_record_dict, f)
f.close()

In [331]:
K_record_dict['PD4107a']['chr6']['record']

{0: ({'powerlaw': (array([-0.32125907,  1.16890521]), 0.04742939351431067)},
  {'expon': (array([-3.69835674e-07,  5.47381262e-01]), 0.17701086160806853)}),
 1: ({'powerlaw': (array([-0.37297023,  1.35705693]), 0.05506382066217318)},
  {'expon': (array([-3.98651583e-07,  5.92229940e-01]), 0.18849200420262494)}),
 2: ({'powerlaw': (array([-0.31759513,  1.16193177]), 0.04176685917469377)},
  {'expon': (array([-6.19802212e-07,  6.44123142e-01]), 0.2097316156629595)}),
 3: ({'powerlaw': (array([-0.31272949,  1.06328925]), 0.011937831127169226)},
  {'expon': (array([-1.09121118e-06,  8.12148826e-01]), 0.22089497608761807)}),
 4: ({'powerlaw': (array([-1.26185951,  3.12027123]), 7.850462293418876e-17)},
  {'expon': (array([-8.07895245e-07,  7.16782153e-01]), 0.27851166626135776)}),
 5: ({'powerlaw': (array([-0.68260619,  1.70536226]), 3.925231146709438e-17)},
  {'expon': (array([-1.30111094e-06,  8.45867429e-01]), 0.27034532835701836)}),
 6: ({'powerlaw': (array([-0.5       ,  1.21568188]), 

In [90]:
nk_record_dict = {}

for patient in tqdm.tqdm(Nk_dict.keys()):
    nk_record_dict[patient] = {}
    patient_id = patient
    df_1 = df_breast[(df_breast[0] == patient_id) & (df_breast[1]=='subs')]
    print('=======> now is sliding <=======')
    distance, df_bucket = lambda_sliding(df_1,2,3,4,50000000,7000000,autosomal=True)
    for c in Nk_dict[patient_id]:
        nk_record_dict[patient][c] = {}
        print('=======> now is fitting <=======')
        record_dict, threshold_list = calculations(df_breast, patient_id, distance, df_bucket, c)
        nk_record_dict[patient][c]['record'] = record_dict
        nk_record_dict[patient][c]['theta'] = threshold_list

  0%|                                                     | 0/8 [00:00<?, ?it/s]

=======> now is sliding <=======
=======> now is fitting <=======


/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1760: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=d

=======> now is sliding <=======
=======> now is fitting <=======


/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
 25%|███████████                                 | 2/8 [09:09<28:27, 284.66s/it]

...

/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/stats/_distn_infrastructure.py:1760: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/mhy/Library/Python/3.7/lib/python/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/mhy/Library/Python/3.7/lib/python/site-pac

=======> now is sliding <=======
=======> now is fitting <=======


 88%|██████████████████████████████████████▌     | 7/8 [32:50<05:00, 300.72s/it]

=======> now is sliding <=======
=======> now is fitting <=======


100%|████████████████████████████████████████████| 8/8 [40:44<00:00, 305.62s/it]


In [91]:
f = open('/Users/mhy/Desktop/Project/AlexandrovEtAI/Round2Results/NonKataegis_StdEr.pkl', 'wb')
pickle.dump(nk_record_dict, f)
f.close()

### conclude results

In [308]:
def stat_summary(record_dict, regime=0, std_erlst=[], est_palst=[], dict_EstStd = {}):
    for k in record_dict.keys():
        if list(record_dict[k][regime].keys())[0] == 'powerlaw':
            std_error = list(record_dict[k][regime].values())[0][1]
            if np.isnan(std_error)==True:
                continue
            std_erlst = std_erlst + [std_error] # value list
            est_param = list(record_dict[k][regime].values())[0][0][0]
            if np.isnan(est_param)==True:
                continue
            est_param = -1*est_param
            est_palst = est_palst + [est_param]
            
    std_mean = sum(std_erlst)/len(std_erlst)
    est_mean = sum(est_palst)/len(est_palst) # single value.
    dict_EstStd['std_mean'] = std_mean
    dict_EstStd['est_mean'] = est_mean
    dict_EstStd['std_erlst'] = std_erlst
    dict_EstStd['est_palst'] = est_palst
        
    return dict_EstStd

In [309]:
over_dict = {}
for patient in tqdm.tqdm(k_dict.keys()):
    over_dict[patient]={}
    for chro in k_dict[patient]:
        temp = K_record_dict[patient][chro]['record']
        temp_dict = stat_summary(temp)
        over_dict[patient][chro] = copy.copy(temp_dict)

100%|███████████████████████████████████████████| 5/5 [00:00<00:00, 1267.77it/s]


In [310]:
for patient in over_dict.keys():
    for chro in over_dict[patient].keys():
        print(patient,'_',chro,'has est:',over_dict[patient][chro]['est_mean'],'and std:',over_dict[patient][chro]['std_mean'])

PD7409a _ chr8 has est: 0.7355510075033269 and std: 0.11683330100288732
PD7404a _ chr17 has est: 1.0793228717478942 and std: 0.21684797117084983
PD7404a _ chr1 has est: 0.8244942847599759 and std: 0.14356851504536305
PD4199a _ chr5 has est: 0.4126862951368917 and std: 0.05005546298340019
PD4199a _ chr8 has est: 0.513141433639735 and std: 0.15202944800800675
PD4199a _ chr1 has est: 0.4888912718263096 and std: 0.11418061344536402
PD4103a _ chr3 has est: 0.7246119505771703 and std: 0.15282790069768443
PD4103a _ chr4 has est: 0.5313036169209419 and std: 0.1553716646594302
PD4103a _ chr8 has est: 0.504712828999789 and std: 0.12534479922211833
PD4103a _ chr12 has est: 0.7798143202932045 and std: 0.15406845443883943
PD4107a _ chr6 has est: 0.7873897760311082 and std: 0.09501203122368565


In [311]:
# generate table
NK_over_dict = {}
for patient in tqdm.tqdm(Nk_dict.keys()):
    NK_over_dict[patient]={}
    for chro in Nk_dict[patient]:
        temp_dict = nk_record_dict[patient][chro]['record']
        temp_connk = stat_summary(temp_dict)
        NK_over_dict[patient][chro] = copy.copy(temp_connk)

100%|███████████████████████████████████████████| 8/8 [00:00<00:00, 3947.12it/s]


In [312]:
for patient in NK_over_dict.keys():
    for chro in NK_over_dict[patient].keys():
        print(patient,'_',chro,'has est:',NK_over_dict[patient][chro]['est_mean'],'and std:',NK_over_dict[patient][chro]['std_mean'])

PD6409a _ chr10 has est: 0.019793690609768433 and std: 0.0018871156894279717
PD6043a _ chr6 has est: 0.36218132516315904 and std: 0.16190605799536267
PD4109a _ chr5 has est: 0.07382659172314912 and std: 0.0
PD4224a _ chr6 has est: 0.04293087065145992 and std: 0.017139405980741058
PD4072a _ chr12 has est: 0.38266432733632705 and std: 0.13128790819262084
PD4955a _ chr3 has est: 0.26540860052742854 and std: 0.00801377632020183
PD4607a _ chr5 has est: 0.5912860541039621 and std: 0.17904498989237466
PD4120a _ chr8 has est: 0.7396974326356007 and std: 0.18281673658947653
